In [ ]:
from copy import deepcopy
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from src.jss_lite.jss_lite import jss_lite
import numpy as np
import time
class tree_node:
    def __init__(self, action=None,parent=None):
        self.action=action
        self.children=[]
        self.state=None
        self.parent=parent
        self.observation=None
        self.visited=False
        # extra attributes for search:
        self.leaf_closed=False
class tree_construct:
    def __init__(self):

        curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
        inst=curr_dir + '/resources/jsp_instances/standard/la01.txt'
        instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
        self.env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
        # create root
        self.root=tree_node()
        # get initial state into root; todo: change for multi instances, create all childs for instances
        self.root.observation=self.env.reset()
        self.root.state=self.env.get_state()
        self.done=False
        self.reward=0

        # path management
        # open leafs should store the leafs with highest rating
        self.open_leafs=[]
        self.open_leafs.append(self.root)
        self.current_path=self.root
        self.sol_pathes=None
    def insert(self,action=None,parent=None):
        parent.children.append(tree_node(action=action,parent=self))
        self.env.set_state(parent.state)
        parent.children[-1].observation,reward,done,info=self.env.step(action)
        parent.children[-1].state=self.env.get_state()
        if done:
            self.__on_done(reward)

    def __on_done(self,reward):
        self.done=True
        if reward > self.reward:
            self.reward=reward

    def solve(self,max_pathes=None,max_time=None,threshold=0.4):
        # here comes the solving method with abort conditions
        start_time=time.time()
        p_time=time.time()
        self.sol_pathes=0

        if max_pathes==None:
            max_pathes=1000 
        if max_time==None:
            max_time=60
        #print(self.open_leafs)
        while self.open_leafs:
            self.sol_pathes+=1
            # abort conditions:
            node=self.open_leafs.pop()
            if self.sol_pathes > max_pathes or time.time()-start_time > max_time:
                return False
            while not self.done:
                loop=0
                n_r=np.random.uniform(0,1)
                if  n_r < 0.05:
                    #print(3)
                    p_r=3
                elif n_r < 0.1:
                    #print(2)
                    p_r=2
                else:
                    p_r=1
                possible_actions=np.random.choice(len(node.observation['action_mask']), p_r, p=(node.observation['action_mask'] / node.observation['action_mask'].sum()))
                
                for action in set(possible_actions):
                    self.insert(action=action,parent=node)
                for child in node.children:
                    if child != node.children[-1]:
                        #todo: check here if in action mask
                        self.open_leafs.append(child)
                while True:
                    if node.children[-1].visited==False:
                        node.children[-1].visited=True
                        node=node.children[-1]
                        
                        break
                    loop+=1
                    if loop > len(node.children):
                        print("all childrens visited; return done equal true")
                        self.done=True
            self.done=False
            




In [ ]:
init=tree_construct()
init.solve()
print(init.reward)
print(init.sol_pathes)